### Create a df to sum up all results from all classif
    0-Good/Bad 1-Poor/Partial 2-Partial/Detailed 3-Poor/Detailed

In [6]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd
import numpy as np
from itertools import product

conds = {'0':['good','bad'],'1':['poor','partial'],'2':['partial','detailed'],
         '3':['poor','detailed']}

for cond in conds:
    ###############################################################################
    path = r'/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/'
    path_npz = join(path, 'figures_npz/1_'+conds[cond][0]+'_'+conds[cond][1]+'_4500_expi_noart/')
    path_mask = join(path_npz, 'masks_visbrain/')
    path_to_save = join(path, 'Bilan_classif/0_all_elecs_phase/')
    npz_form = join(path_npz,'{}_sources_phase_{}_odor_'+conds[cond][0]+'_'+conds[cond][1]+'_Expi_sel_phys.npz')
    masks_vis_form = join(path_mask, '{}_mask_min{}_{}_minwin{}_th{}.npy')
    ###############################################################################
    if not exists(path_to_save):
        makedirs(path_to_save)
    ###############################################################################

    freqs = ['2_theta']#'0_VLFC','1_delta', '3_alpha'

    for freq in freqs:
        #Load all data for one classif one freq
        arch_sig = np.load(npz_form.format('All_subjects',freq))
        print(arch_sig.files,arch_sig['s_time'])
        #Create a df with all electrodes info
        features = ['su_codes','s_elec','s_channels','s_labels','s_aal_RL','s_BA']
        df_elec = np.array([])
        for f in features:
            data = arch_sig[f]
            df_elec = np.vstack((df_elec,data)) if np.size(df_elec) else data
        xyz = arch_sig['s_xyz']
        x,y,z = xyz[:,0],xyz[:,1],xyz[:,2]
        df_elec = np.vstack((df_elec,x,y,z))
        df0 = pd.DataFrame(data=df_elec.T, columns=features+['x','y','z'])
        #Select max values for power and AUC scores and create Time array
        da = arch_sig['s_da'][:,:39]
        pow0_max, pow1_max, idx_max, da_max, time_max = np.array([]), np.array([]), np.array([]), np.array([]),np.array([])
        pow0_std, pow1_std = np.array([]), np.array([])
        for elec in range(da.shape[0]):
            #print('shape da',da.shape)
            idx = [i for i,j in enumerate(da[elec]) if j == max(da[elec])]
            if len(idx) != 1:
                idx = [idx[0]]
            #print('index',idx)
            pow0, pow1 = arch_sig['s_elec_p0'][elec][idx], arch_sig['s_elec_p1'][elec][idx]
            std0, std1 = arch_sig['s_elec_p0_std'][elec][idx], arch_sig['s_elec_p1_std'][elec][idx]
            time = arch_sig['s_time'][idx]
            da_elec = da[elec][idx]
            pow0_max = np.hstack((pow0_max,pow0)) if np.size(pow0_max) else pow0
            pow1_max = np.hstack((pow1_max,pow1)) if np.size(pow1_max) else pow1
            pow0_std = np.hstack((pow0_std,std0)) if np.size(pow0_std) else std0
            pow1_std = np.hstack((pow1_std,std1)) if np.size(pow1_std) else std1
            time_max = np.hstack((time_max,time)) if np.size(time_max) else time
            idx_max = np.hstack((idx_max,idx)) if np.size(idx_max) else idx
            da_max = np.hstack((da_max, da_elec)) if np.size(da_max) else da_elec
        print(pow0_max.shape, pow0_std.shape)
        df_data = np.concatenate((pow0_max[:,np.newaxis],pow1_max[:,np.newaxis],
                da_max[:,np.newaxis],idx_max[:,np.newaxis], time_max[:,np.newaxis],
                pow0_std[:,np.newaxis], pow1_std[:,np.newaxis]),axis=1)
        df1 = pd.DataFrame(data=df_data, columns=['p0','p1','da','idx','time','std0','std1'])
        df_all = pd.concat((df0,df1),axis=1)
        df_name = cond+'_Classif_'+conds[cond][0]+'_'+conds[cond][1]+'_'+freq+'_all_elecs.csv'
        df_all.to_csv(path_to_save+df_name)

['s_da', 's_elec_p0', 's_aal', 's_channels', 's_labels', 's_th_01', 's_BA', 's_aal_RL', 'su_codes', 's_elec_p1', 's_elec', 's_elec_p0_std', 's_elec_p1_std', 's_xyz', 's_th_05', 's_time'] [ 0.34960938  0.44921875  0.54882812  0.6484375   0.74804688  0.84765625
  0.94726562  1.046875    1.14648438  1.24609375  1.34570312  1.4453125
  1.54492188  1.64453125  1.74414062  1.84375     1.94335938  2.04296875
  2.14257812  2.2421875   2.34179688  2.44140625  2.54101562  2.640625
  2.74023438  2.83984375  2.93945312  3.0390625   3.13867188  3.23828125
  3.33789062  3.4375      3.53710938  3.63671875  3.73632812  3.8359375
  3.93554688  4.03515625  4.13476562]
(499,) (499,)
['s_da', 's_elec_p0', 's_aal', 's_channels', 's_labels', 's_th_01', 's_BA', 's_aal_RL', 'su_codes', 's_elec_p1', 's_elec', 's_elec_p0_std', 's_elec_p1_std', 's_xyz', 's_th_05', 's_time'] [ 0.34960938  0.44921875  0.54882812  0.6484375   0.74804688  0.84765625
  0.94726562  1.046875    1.14648438  1.24609375  1.34570312  1.445

### Mask significant results as function of time of the effect (wins)
    statistics p < 0.05 corrected over electrodes and time

In [7]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd
import numpy as np
from itertools import product
###############################################################################################
path = r'/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/'
path_df = join(path, 'Bilan_classif/0_all_elecs_phase/')
path2save = join(path,'Bilan_classif/1_signif_elecs_wins_phase/')
###############################################################################################
if not exists(path2save):
    makedirs(path2save)
##############################################################################################
conds = {'0':['good','bad'],'1':['poor','partial'],'2':['partial','detailed'],
         '3':['poor','detailed']}
freqs = ['0_VLFC','1_delta','2_theta', '3_alpha']
bsls,wins,ths = ['None'], [1,2,3], ['05']

for cond, freq, bsl, th in product(conds, freqs,bsls,ths):
    df = pd.read_csv(path_df+cond+'_Classif_'+conds[cond][0]+'_'+conds[cond][1]+'_'+freq+'_all_elecs.csv')
    path_mask = join(path,'figures_npz/'+'1_'+conds[cond][0]+'_'+conds[cond][1]+'_4500_expi_noart/masks_stat/')
    for i,win in enumerate(wins):
        #load data to plot and mask
        mask = np.load(path_mask+'All_subjects_mask_stat_'+freq+'_minwin'+str(win)+'_th'+th+'.npy')[:,np.newaxis]
        mask = np.repeat(mask,df.shape[1],axis=1)
        if mask.dtype == bool:
            df_sel = df.mask(mask).dropna()
            df_name = cond+'_Classif_'+conds[cond][0]+'_'+conds[cond][1]+'_'+freq+'_win'+str(win)+'.csv'
            df_sel.to_csv(path2save+df_name)

### Mask significant results as function of time and nb of patients
    statistics p < 0.05 corrected over electrodes and time


In [8]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd
import numpy as np
from itertools import product
###############################################################################################
path = r'/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/'
path_df = join(path, 'Bilan_classif/1_signif_elecs_wins_phase/')
path2save = join(path,'Bilan_classif/2_signif_elecs_patients_phase/')
###############################################################################################
if not exists(path2save):
    makedirs(path2save)
##############################################################################################
conds = {'0':['good','bad'],'1':['poor','partial'],'2':['partial','detailed'],
         '3':['poor','detailed']}
freqs = ['0_VLFC','1_delta','2_theta', '3_alpha']
bsls,wins,ths = ['None'], [1,2,3], ['05']

df3, df4, df5 = None, None, None
for cond, freq, bsl, th, win in product(conds, freqs,bsls,ths,wins):
    print('processing on', conds[cond], win, freq)
    df = pd.read_csv(path_df+cond+'_Classif_'+conds[cond][0]+'_'+conds[cond][1]+'_'+freq+'_win'+str(win)+'.csv')
    df = df.drop(['Unnamed: 0.1'], axis=1)
    df = df.rename(index=str,columns={'Unnamed: 0':'old_idx'})
    print(df.shape)
    
    #count and select regions with enough patients
    count = df.groupby(['s_aal_RL']).nunique()
    count = count['su_codes']
    rois_sig2,rois_sig3, rois_sig4, rois_sig5 = [], [], [],[]
    for i,row in count.iteritems():
        #print('how many patients',row, 'for roi', i)
        if row >=2:
            rois_sig2.append(i)
        if row >=3:
            rois_sig3.append(i)
        if row >=4:
            rois_sig4.append(i)
        if row >=5:
            rois_sig5.append(i)
    print(rois_sig2,rois_sig3,rois_sig4,rois_sig5)
    
    # select data only in selected regions 
    df2 = df[df['s_aal_RL'].isin(rois_sig2)] if rois_sig2 else print('No rois2')
    df3 = df[df['s_aal_RL'].isin(rois_sig3)] if rois_sig3 else print('No rois3')
    df4 = df[df['s_aal_RL'].isin(rois_sig4)] if rois_sig4 else print('No rois4')
    df5 = df[df['s_aal_RL'].isin(rois_sig5)] if rois_sig5 else print('No rois5')
    # save data for selected regions
    df2_name = cond+'_Classif_'+conds[cond][0]+'_'+conds[cond][1]+'_'+freq[:]+'_win'+str(win)+'_patients2.csv'
    df3_name = cond+'_Classif_'+conds[cond][0]+'_'+conds[cond][1]+'_'+freq[:]+'_win'+str(win)+'_patients3.csv'
    df4_name = cond+'_Classif_'+conds[cond][0]+'_'+conds[cond][1]+'_'+freq[:]+'_win'+str(win)+'_patients4.csv'
    df5_name = cond+'_Classif_'+conds[cond][0]+'_'+conds[cond][1]+'_'+freq[:]+'_win'+str(win)+'_patients5.csv'
    df2.to_csv(path2save+df2_name) if df2 is not None else print('no df2 saved')
    df3.to_csv(path2save+df3_name) if df3 is not None else print('no df3 saved')
    df4.to_csv(path2save+df4_name) if df4 is not None else print('no df4 saved')
    df5.to_csv(path2save+df5_name) if df5 is not None else print('no df5 saved')

processing on ['good', 'bad'] 1 0_VLFC
(41, 15)
['Frontal Mid', 'Frontal Mid Orb', 'Frontal Sup', 'Frontal Sup Orb', 'Paracentral Lobule', 'Temporal Inf', 'Temporal Mid', 'Temporal Sup'] ['Frontal Mid Orb', 'Temporal Mid'] ['Temporal Mid'] []
No rois5
no df5 saved
processing on ['good', 'bad'] 2 0_VLFC
(10, 15)
['Frontal Mid Orb'] [] [] []
No rois3
No rois4
No rois5
no df3 saved
no df4 saved
no df5 saved
processing on ['good', 'bad'] 3 0_VLFC
(4, 15)
[] [] [] []
No rois2
No rois3
No rois4
No rois5
no df2 saved
no df3 saved
no df4 saved
no df5 saved
processing on ['good', 'bad'] 1 1_delta
(45, 15)
['Cingulum Ant', 'Frontal Inf Orb', 'Frontal Sup', 'Frontal Sup Medial', 'Frontal Sup Orb', 'Hippocampus', 'Postcentral', 'Temporal Mid', 'Temporal Sup'] ['Frontal Inf Orb'] ['Frontal Inf Orb'] []
No rois5
no df5 saved
processing on ['good', 'bad'] 2 1_delta
(4, 15)
[] [] [] []
No rois2
No rois3
No rois4
No rois5
no df2 saved
no df3 saved
no df4 saved
no df5 saved
processing on ['good', 'bad']